# Week 1: Adding unit model to flowsheet

This tutorial will go through the basic workflow to create a WaterTAP model with a single unit proces (pump).

1. Create model
2. Add flowsheet
3. Add property model
4. Add pump model
5. Set operating conditions
6. Scaling model
7. Initialize model
8. Solve model

## Step 0: Import necessary packages

### Any WaterTAP model will begin with importing the relevant components from Pyomo, IDAES, and WaterTAP.

From Pyomo:
- Concrete model object
- units 
- Helper function(s) to check solver status

From IDAES:
- Flowsheet block
- Helper function to check degrees of freedom

From WaterTAP:
- Property models
- Unit models
- Solver

> Both `ConcreteModel` and `FlowsheetBlock` are required imports for any WaterTAP model.


In [15]:
# Pyomo imports
from pyomo.environ import (
    ConcreteModel,
    check_optimal_termination,
    assert_optimal_termination,
    units as pyunits,
)

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump


## Step 1: Create base model

Pyomo's `ConcreteModel` is the basis for any WaterTAP model.

No arguments are required. Convention is to call this `m` but any local name can be used.

In [16]:
m = ConcreteModel()

## Step 2: Add flowsheet block

The [`FlowsheetBlock`](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/flowsheet_block.html#) from IDAES forms the base for all WaterTAP system models.

All model components (unit models, property models, variables, constraints, etc.) are built upon the flowsheet block.

The flowsheet block will create the time domain for the flowsheet. WaterTAP is a steady-state modeling framework, so we set `dynamic=False` when creating the `FlowsheetBlock`.

(Note: `dynamic=False` is the default argument, but excluding it will raise a warning.)

In [17]:
m.fs = FlowsheetBlock(dynamic=False)

# Step 3: Add property model

With the flowsheet created, we can add any sub-models required for the WaterTAP system model.

The property model used will be dictated by the models on the flowsheet. In this demonstration, we are using the WaterTAP [seawater property model](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/seawater.html).

Creating the `SeawaterParameterBlock` does not require any arguments. However, other property models can have many required and optional arguments to specify the model.

In [18]:
m.fs.properties = SeawaterParameterBlock()

# Step 4: Add unit model

After adding the property model, we can add the unit models to the flowsheet. In this demonstration, we are adding a single [pump model](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/pump.html).

Adding any unit model requires specifying the property package via the `property_package` argument. Here, we are specifying that the property model to be used with our pump model is `m.fs.properties` that we just added to the flowsheet.

In [19]:
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Step 5: Set operating conditions

Now that we have added all the necessary models to the flowsheet, we can specify the operating conditions.

First, we check the degrees of freedom that need to be specified prior to solving the model.


In [20]:
print("Degrees of freedom =", degrees_of_freedom(m))

Degrees of freedom = 6


The degrees of freedom for the pump model include 
- the change in pressure from inlet to outlet
- pump efficiency
- inlet stream state variables

The inlet stream conditions to any unit model are defined by the property model state variables. For the pump model, the inlet and outlet properties are managed by a [control volume](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/control_volume_0d.html#module-idaes.core.base.control_volume0d).

We can use the `.display()` method on the pump unit model control volume. Using this method on any block will automatically go through all the sub-blocks and show all the variables, constraints, and objectives currently on each block.

In [21]:
m.fs.pump.control_volume.display()

Block fs.pump.control_volume

  Variables:
    work : Work transferred into control volume
        Size=1, Index=fs._time, Units=kg*m**2/s**3
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :  None :   0.0 :  None : False : False :  Reals
    deltaP : Pressure difference across unit
        Size=1, Index=fs._time, Units=kg/m/s**2
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :  None :   0.0 :  None : False : False :  Reals

  Objectives:
    None

  Constraints:
    material_balances : Size=2
        Key          : Lower : Body : Upper
        (0.0, 'H2O') :   0.0 :  0.0 :   0.0
        (0.0, 'TDS') :   0.0 :  0.0 :   0.0
    pressure_balance : Size=1
        Key : Lower : Body : Upper
        0.0 :   0.0 :  0.0 :   0.0
    isothermal_balance : Size=1
        Key : Lower : Body : Upper
        0.0 :   0.0 :  0.0 :   0.0

  Blocks:
    Block fs.pump.control_volume.properties_in[0.0]
    
      Variables:
        flow_mass_phase_comp : 

This reveals that upon `m.fs.pump.control_volume` are two sub-blocks called `properties_in` and `properties_out`. Each of these are indexed to `0`, which is the default time-index for steady-state models.

To define the inlet stream to our pump, we only need to specify the state variables for `properties_in`. The state variables are defined by the property model used for the unit model (`SeawaterParameterBlock` here) but are nearly always:
- `pressure`: pressure of inlet stream (Pa)
- `temperature`: temperature of (K)
- `flow_mass_phase_comp["Liq", "H2O"]`: mass flow rate of water (kg/s)
- `flow_mass_phase_comp["Liq", "TDS"]`: mass flow rate of TDS (kg/s)

> Note: The naming of property variables in WaterTAP property models follow the [naming conventions used in IDAES](https://idaes-pse.readthedocs.io/en/latest/explanations/conventions.html#standard-naming-format).

In [22]:
m.fs.pump.control_volume.properties_in[0].pressure.fix(101325)  # Pa
m.fs.pump.control_volume.properties_in[0].temperature.fix(298.15)  # K
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(1) # kg/s
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.1) # kg/s

print("Degrees of freedom =", degrees_of_freedom(m))

Degrees of freedom = 2


With the inlet stream specified, the remaining two degrees of freedom are variables that are built directly on the pump block itself (rather than on the control volume sub-block):

In [23]:
m.fs.pump.deltaP.fix(500000)  # Pa
m.fs.pump.efficiency_pump.fix(0.8)

print("Degrees of freedom =", degrees_of_freedom(m))

Degrees of freedom = 0


In [ ]:
m.fs.pump.display()

In [ ]:
m.fs.pump.control_volume.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", "Liq"): 1,
        ("conc_mass_phase_comp", ("Liq", "TDS")): 0.5,
        ("pressure", None): 101325,
        ("temperature", None): 298,
    },
    hold_state=True,
)

# m.fs.feed.properties.calculate_state(
#         var_args={
#             ("flow_vol_phase", "Liq"): flow_in,  # m3/s
#             ("conc_mass_phase_comp", ("Liq", target_ion)): conc_mass_in,  # kg/m3
#             ("pressure", None): 101325,
#             ("temperature", None): 298,
#         },
#         hold_state=True,
#     )

In [ ]:
# Provide unit model inputs. For the pump we need to provide inlet flow, inlet pressure, outlet pressure, temperature and efficiency

# Add pyunits

feed_flow_mass = 1 # kg/s
feed_mass_frac_TDS = 0.035
feed_mass_frac_H2O = 1 - feed_mass_frac_TDS

m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "TDS"].fix(
    feed_flow_mass * feed_mass_frac_TDS
)
m.fs.pump.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    feed_flow_mass * feed_mass_frac_H2O
)

feed_pressure_in = 1e5 # Pa
feed_pressure_out = 5e5 # Pa
feed_temperature = 273.15 + 25 # K

m.fs.pump.inlet.pressure[0].fix(feed_pressure_in)
m.fs.pump.inlet.temperature[0].fix(feed_temperature)
m.fs.pump.outlet.pressure[0].fix(feed_pressure_out)

m.fs.pump.efficiency_pump.fix(0.75)

In [ ]:
m.fs.pump.inlet.display()

In [ ]:
# Lets check the degrees of freedom of the flowsheet! This should ideally be zero now.
degrees_of_freedom(m)

In [ ]:
# Let's initialize the flowsheet
m.fs.pump.initialize()

In [ ]:
# Let's solve the flowsheet now!
solver = get_solver()
results = solver.solve(m, tee=False)
# Ensure that the solver status is ok and optimal termination
assert_optimal_termination(results)

In [ ]:
def build():
    return m

m.fs.pump.properties_in.calculate_state(
    var_args={
        ("flow_vol_phase", ["Liq"]): m.fs.pump.inlet.flow_vol_phase[0, "Liq"],
        ("conc_mass_phase_comp", ["Liq", "TDS"]): m.fs.pump.inlet.conc_mass_phase_comp[0, "Liq", "TDS"],
        ("pressure"): feed_pressure_in,
        ("temperature"): feed_temperature,
    }
)